In [ ]:
###### VACATION MODE ##############

import cv2
from ultralytics import YOLO
import winsound
import threading
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

password = "***************"  # Your decrypted password from gmail here
from_email = "*******************"
to_email = "******************"

server = smtplib.SMTP('smtp.gmail.com: 587')
server.starttls()
server.login(from_email, password)

def send_email(to_email, from_email, object_detected=1):
    message = MIMEMultipart()
    message['From'] = from_email
    message['To'] = to_email
    message['Subject'] = "Security Alert"
    message_body = f'ALERT - {object_detected} individual(s) have been detected!!'
    message.attach(MIMEText(message_body, 'plain'))
    server.sendmail(from_email, to_email, message.as_string())

video = cv2.VideoCapture(0)
modelo = YOLO('yolov8n.pt')

area = [100, 190, 1150, 700] 

alarmeCtl = False
recording = False
out = None
start_time = None
end_time = None
record_folder = ' ' #Records folder path

def alarme():
    global alarmeCtl
    for _ in range(7):
        winsound.Beep(2500, 500)

    alarmeCtl = False

def start_recording(img):
    global record_folder
    recording['value'] = True
    start_time = time.time()
    fourcc = cv2.VideoWriter_fourcc(*'H264')
    filename = os.path.join(record_folder, f"REC_{time.strftime('%Y-%m-%d_%H-%M-%S')}.mp4")
    out['value'] = cv2.VideoWriter(filename, fourcc, 10, (img.shape[1], img.shape[0]))


def stop_recording():
    global recording, out, end_time
    recording = False
    end_time = time.time()
    out.release()

while True:
    check, img = video.read()
    img = cv2.resize(img, (1270, 720))
    img2 = img.copy()
    cv2.rectangle(img2, (area[0], area[1]), (area[2], area[3]), (0, 255, 0), -1)
    resultado = modelo(img)

    object_detected = False

    for objetos in resultado:
        obj = objetos.boxes
        for dados in obj:
            x, y, w, h = dados.xyxy[0]
            x, y, w, h = int(x), int(y), int(w), int(h)
            cls = int(dados.cls[0])
            cx, cy = (x + w) // 2, (y + h) // 2
            if cls == 0:
                cv2.rectangle(img, (x, y), (w, h), (255, 0, 0), 5)

                if cx >= area[0] and cx <= area[2] and cy >= area[1] and cy <= area[3]:
                    cv2.rectangle(img2, (area[0], area[1]), (area[2], area[3]), (0, 0, 255), -1)
                    cv2.rectangle(img, (100, 30), (470, 80), (0, 0, 255), -1)
                    cv2.putText(img, 'INTRUDER DETECTED', (105, 65), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
                    object_detected = True
                    if not alarmeCtl:
                        alarmeCtl = True
                        threading.Thread(target=alarme).start()

                    if not recording:
                        threading.Thread(target=start_recording).start()

                    threading.Thread(target=send_email, args=(to_email, from_email, len(obj))).start()

            else:
                object_detected = False

    if recording and object_detected:
        end_time = time.time()

    if recording and not object_detected and end_time is not None and time.time() - end_time >= 10:
        threading.Thread(target=stop_recording).start()

    if recording and out is not None:
        out.write(img)

    imgFinal = cv2.addWeighted(img2, 0.5, img, 0.5, 0)

    cv2.imshow('img', imgFinal)
    key = cv2.waitKey(1)
    if key == 27:
        break

video.release()
if recording and out is not None:
    out.release()
cv2.destroyAllWindows()

In [ ]:
###### REGULAR MODE ##############
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

def reg_cam():
    model = YOLO("yolov8n-seg.pt")  # segmentation model
    names = model.model.names
    cap = cv2.VideoCapture(0)
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    while True:
        ret, im0 = cap.read()
        if not ret:
            print("Not working")
            break

        results = model.predict(im0)
        annotator = Annotator(im0, line_width=2)

        if results[0].masks is not None:
            clss = results[0].boxes.cls.cpu().tolist()
            masks = results[0].masks.xy
            for mask, cls in zip(masks, clss):
                annotator.seg_bbox(mask=mask,
                                   mask_color=colors(int(cls), True),
                                   det_label=names[int(cls)])

        #out.write(im0)
        cv2.imshow("instance-segmentation", im0)

        key = cv2.waitKey(1)
        if key == 27:
            break

    #out.release()
    cap.release()
    cv2.destroyAllWindows()

# Call the function
reg_cam()
